In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from docling.datamodel.pipeline_options_vlm_model import ResponseFormat

from archaeo_super_prompt.dataset.load import MagohDataset
from archaeo_super_prompt.pdf_to_text.stream_ocr import process_documents, converter, ollama_vlm_options

# 🦆📃 PDF complete ingestion with Docling preprocessing

We try the young framework Docling and its usage of VLLM on the HuggingFace repositories to achieve thoses tasks:

- document OCR with Italian language analysis (VLLM)
- document chunking with these features:
    - layout-aware
    - smart tokenization

All of these things are possible with incorporating several open ML models into the Docling pipeline

In [ ]:
MAX_SAMPLES_FETCHED = 300
SEED = 0.5

dataset = MagohDataset(MAX_SAMPLES_FETCHED, SEED, True)
_selected_ids = [
    35983
]
selected_ids = set(_selected_ids)
inputs = dataset.get_files_for_batch(selected_ids)

In [ ]:
inputs["filepath"].tolist()

In [ ]:
def main():
    logging.basicConfig(level=logging.DEBUG)
    
    # Example using the Granite Vision model with Ollama:
    vlm_options = ollama_vlm_options(
        model="granite3.2-vision:latest",
        prompt="OCR the full page to markdown.",
        response_format=ResponseFormat.MARKDOWN  # TODO: try doctags
    )
    doc_converter = converter(vlm_options)
    results = process_documents(inputs["filepath"].tolist(), doc_converter)
    return results

In [ ]:
result = main()

In [ ]:
from IPython.display import Markdown
Markdown(result.document.export_to_markdown())